# Data manipulation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy, sklearn, statsmodels
from collections import Counter

## pd.read_csv
Read the data in the `gapminder.csv` file, with the `pd.read_csv` function.

In [ ]:
d = pd.read_csv( "gapminder.csv" )
d

## shape, columns, index, describe, head, tail, min
Look at the `shape`, `columns`, `index` attributes

In [ ]:
d.shape

In [ ]:
d.columns

In [ ]:
d.index

What does the `describe()` method return?

In [ ]:
d.describe()

Use the `head()` and `tail()` methods to look at the first and last 10 rows

In [ ]:
d.head(10)

In [ ]:
d.tail(10)

Use the `min()` and `max()` methods to find the first and last year with data

In [ ]:
d['year'].min(), d['year'].max()

## Indexing
Extract the numeric columns (using their names)

In [ ]:
d.head()

In [ ]:
d[['year', 'lifeExp', 'pop', 'gdpPercap']]

Using `iloc`, extract the first row

In [ ]:
d.iloc[0,:]

Using `iloc`, extract the last 5 values of the last column

In [ ]:
d.iloc[-5:,-1]

Using `loc`, extract the "pop" column

In [ ]:
d.loc[:, 'pop']

What are the minimum and maximum values of the `lifeExp` column?

In [ ]:
d['lifeExp'].min(), d['lifeExp'].max()

## Counting
Use the `unique()` method to find for which year we have data.

In [ ]:
d['year'].unique()

Use the `len()` function to find how many countries there are.

In [ ]:
len( d['country'].unique() )

## Boolean indexing
Extract the rows where `lifeExp` is below 25.

In [ ]:
i = d['lifeExp'] < 25
d[i]


Extract the rows where `country` is "China".

In [ ]:
i = d['country'] == "China"
d[i]

Which countries had a life expectancy below 50 years and a GDP per capita above 10,000?

In [ ]:
i1 = d['lifeExp'] < 50
i2 = d['gdpPercap'] > 10_000
d[i1 & i2]

## sort_values
Where and when was the lowest life expectancy of all times?

In [ ]:
lowest_life_exp = d['lifeExp'].min()
i = d['lifeExp'] == lowest_life_exp
d[i]

In [ ]:
# Other solution (but this assumes there is only one row)
d.sort_values('lifeExp').head(1)

For the latest year, find the 5 countries with the largest GDP per capita.

In [ ]:
latest_year = d['year'].max()
i = d['year'] == latest_year
d[i].sort_values('gdpPercap', ascending=False).head(5)

## groupby
How many countries are there in each continent?

In [ ]:
d.groupby(['continent','year']).size().reset_index().pivot( columns='year', index='continent', values=0 )

In [ ]:
# Other solution
d.groupby('continent')['country'].nunique()

Is the number of countries the same each year?

In [ ]:
d.groupby('year')['country'].count()

What is the total population each year?

In [ ]:
d.groupby('year')['pop'].sum() / 1e9  # In billion

## pivot
Use the `pivot()` method to build a data-frame with one row per country, one column per date, and containing the life expectancy

In [ ]:
d.pivot( index='country', columns='year', values='lifeExp' )

What happens if you call the `melt()` method on that data-frame?

In [ ]:
d.pivot( index='country', columns='year', values='lifeExp' ).melt( ignore_index = False ).reset_index()

## More exercises
What is the average life expectancy for each continent and each year; use `pivot` to present the result one row per year, one column per continent.

In [ ]:
d.groupby(['continent','year'])['lifeExp'].mean().reset_index().pivot( columns='year', index='continent', values='lifeExp' ).round().astype(int)

Are there countries whose life expectancy decreased over time?

In [ ]:
changes = d.pivot( index = 'year', columns = 'country', values = 'lifeExp' ).diff().dropna()
i = np.where( ( changes < 0 ).sum() > 0 )[0]
changes = changes.iloc[:,i].T.round(1)
changes

In [ ]:
# I asked the AI: 
#   "In the following data-frame, highlight negative values in bold and with a pink background; only show one decimal place."
def highlight_negatives(val):
    if val < 0:
        return 'background-color: pink; font-weight: bold;'
    return ''
changes.style.map(highlight_negatives).format("{:.1f}")

With the `diff()` method, compute the change in world population over time

In [ ]:
d.groupby('year')['pop'].sum().diff().dropna() / 1e6  # In million

Compute the change in world GDP over time

In [ ]:
d['GDP'] = d['pop'] * d['gdpPercap']
d.groupby('year')['GDP'].sum().diff().dropna() / 1e12  # In trillion

Which other questions could you ask and answer about this dataset (feel free to ask ChatGPT to come up with such questions)?

In [ ]:
# INSERT TEXT AND CODE

## ChatGPT Exercises

1. What is the average life expectancy for each continent across all years?

In [ ]:
d.groupby(['continent','year'])['lifeExp'].mean().reset_index().pivot( columns='year', index='continent', values='lifeExp' ).round().astype(int)

In [ ]:
# We may prefer a population-weighted average life expectancy.
d['pop*lifeExp'] = d['pop'] * d['lifeExp']
numerator = d.groupby(['continent','year'])['pop*lifeExp'].sum().reset_index().pivot( columns='year', index='continent', values='pop*lifeExp' )
denominator = d.groupby(['continent','year'])['pop'].sum().reset_index().pivot( columns='year', index='continent', values='pop' )
( numerator / denominator ).round().astype(int)


2. Which country had the highest GDP per capita in 2007?

In [ ]:
i = d['year'] == 2007
d[i].sort_values('gdpPercap', ascending=False).head(1)

3. What is the total population of each continent in the most recent year available?

In [ ]:
latest_year = d['year'].max()
i = d['year'] == latest_year
d[i].groupby('continent')['pop'].sum() / 1e9  # In billion

In [ ]:
# Other solution
d.groupby(['continent','year'])['pop'].sum().reset_index().pivot( index = 'year', columns = 'continent', values = 'pop' ).tail(1) / 1e9  # In billion

4. How has the life expectancy in a specific country changed over time?

In [ ]:
country = 'China'
i = d['country'] == country
d[i]


5. Which continent has seen the greatest improvement in life expectancy from 1950 to 2007?

In [ ]:
d['pop*lifeExp'] = d['pop'] * d['lifeExp']
numerator = d.groupby(['continent','year'])['pop*lifeExp'].sum().reset_index().pivot( columns='year', index='continent', values='pop*lifeExp' )
denominator = d.groupby(['continent','year'])['pop'].sum().reset_index().pivot( columns='year', index='continent', values='pop' )
lifeExp = ( numerator / denominator )
( lifeExp[2007] - lifeExp[1952] ).sort_values(ascending=False).head(1)

6. What are the top 5 countries with the largest population growth between two years?

In [ ]:
year1 = 1952
year2 = 2007
d.pivot( index = 'country', columns = 'year', values = 'pop' )[[year1, year2]].T.diff().dropna().T.sort_values(year2, ascending=False).head(5)

In [ ]:
# We may prefer the relative growth
log_population = np.log( 
    d.pivot( index = 'country', columns = 'year', values = 'pop' )
)
growth = log_population[[year1, year2]].T.diff().dropna().T.sort_values(year2, ascending=False)
( 100 * np.expm1( growth ).head(5) ).round().astype(int)  # In percent


7. How is GDP per capita related to life expectancy for different countries?

In [ ]:
# We will see that tomorrow, when we learn how to plot data.

8. What was the median life expectancy for each continent in 1980?

In [ ]:
i = d['year'] == 1982
d[i].groupby('continent')['lifeExp'].median()

9. Which countries had a life expectancy above 75 years in 2007?

In [ ]:
i1 = d['year'] == 2007
i2 = d['lifeExp'] > 75
d[i1 & i2]

10. How has the population of each continent changed over time?

In [ ]:
d.groupby(['continent','year'])['pop'].sum().reset_index().pivot( index = 'year', columns = 'continent', values = 'pop' ) / 1e9  # In billion

## Extra exercise (difficult)

Extract population, GDP per capita, and life expectancy (and more, if you want) from  `https://github.com/open-numbers/ddf--gapminder--systema_globalis`.

Is it the same data as before? What are the differences?

In [ ]:
! git clone https://github.com/open-numbers/ddf--gapminder--systema_globalis

In [ ]:

directory = "ddf--gapminder--systema_globalis"
d1 = pd.read_csv( f"{directory}/countries-etc-datapoints/ddf--datapoints--gdppercapita_us_inflation_adjusted--by--geo--time.csv" )
d2 = pd.read_csv( f"{directory}/countries-etc-datapoints/ddf--datapoints--total_population_with_projections--by--geo--time.csv" )
d3 = pd.read_csv( f"{directory}/countries-etc-datapoints/ddf--datapoints--life_expectancy_at_birth_data_from_ihme--by--geo--time.csv" )
d4 = pd.read_csv( f"{directory}/ddf--entities--geo--country.csv" )

d0 = ( 
    d1
    .merge( d2, on = ['geo', 'time'], how = 'outer' )
    .merge( d3, on = ['geo', 'time'], how = 'outer' )
    .merge( d4, left_on = 'geo', right_on = 'country', how = 'left' )
)
d0[['name', 'time', 'gdppercapita_us_inflation_adjusted', 'total_population_with_projections', 'life_expectancy_at_birth_data_from_ihme', 'world_4region']]
